<a href="https://colab.research.google.com/github/imyoungchae/Tensorflow_study/blob/main/tensorflow9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Individual House Hold Electric Power Consumption Dataset을 활용한 예측

# Import

In [3]:
import urllib
import os
import zipfile
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

# 데이터셋 다운로드

In [4]:
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()

In [5]:
download_and_extract_data()

In [6]:
df = pd.read_csv('household_power_consumption.csv', sep=',', infer_datetime_format=True, index_col='datetime', header=0)
df.head(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
2006-12-16 17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
2006-12-16 17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
2006-12-16 17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
2006-12-16 17:32:00,3.668,0.510,233.99,15.8,0.0,1.0,17.0


# 데이터 정규화

In [7]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [8]:
# FEATURES에 데이터프레임의 Column 개수 대입
N_FEATURES = len(df.columns)

# 데이터프레임을 numpy array으로 가져와 data에 대입
data = df.values

# 데이터 정규화
data = normalize_series(data, data.min(axis=0), data.max(axis=0))
data

array([[0.43377912, 0.47826087, 0.04036551, ..., 0.        , 0.01282051,
        0.85      ],
       [0.55716135, 0.49885584, 0.0355582 , ..., 0.        , 0.01282051,
        0.8       ],
       [0.55867127, 0.56979405, 0.03420739, ..., 0.        , 0.02564103,
        0.85      ],
       ...,
       [0.03710095, 0.        , 0.05983313, ..., 0.        , 0.        ,
        0.        ],
       [0.03559103, 0.        , 0.06515693, ..., 0.        , 0.        ,
        0.        ],
       [0.03774806, 0.        , 0.06730234, ..., 0.        , 0.01282051,
        0.        ]])

In [9]:
pd.DataFrame(data).describe()

,0,1,2,3,4,5,6
count,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000
mean,0.156411,0.147141,0.064697,0.152278,0.016950,0.024085,0.375711
std,0.144040,0.134578,0.013900,0.139343,0.086787,0.097022,0.433595
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.021786,0.000000,0.055344,0.024752,0.000000,0.000000,0.000000
50%,0.131795,0.132723,0.065713,0.123762,0.000000,0.000000,0.000000
75%,0.239431,0.224256,0.074652,0.227723,0.000000,0.012821,0.850000
max,0.979077,1.000000,0.107350,0.980198,1.000000,1.000000,1.000000


# 데이터 분할

In [10]:
# 데이터셋 분할 (0.8). 
# 기존 0.5 -> 0.8로 변경 // 다른 비율로 변경 가능
split_time = int(len(data) * 0.8)

In [11]:
x_train = data[:split_time]
x_valid = data[split_time:]

# Windowed Dataset 생성

In [12]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=(n_past + n_future), shift = shift, drop_remainder = True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.shuffle(len(series))
    ds = ds.map(
        lambda w: (w[:n_past], w[n_past:])
    )
    return ds.batch(batch_size).prefetch(1)

In [13]:
# 다음 4개의 옵션은 주어짐
BATCH_SIZE = 32 # 변경 가능
N_PAST = 24 # 변경 불가.
N_FUTURE = 24 # 변경 불가.
SHIFT = 1 # 변경 불가.

In [14]:
train_set = windowed_dataset(series=x_train, 
                             batch_size=BATCH_SIZE,
                             n_past=N_PAST, 
                             n_future=N_FUTURE,
                             shift=SHIFT)

valid_set = windowed_dataset(series=x_valid, 
                             batch_size=BATCH_SIZE,
                             n_past=N_PAST, 
                             n_future=N_FUTURE,
                             shift=SHIFT)

# 모델 생성

In [15]:
model = tf.keras.models.Sequential([
    Conv1D(filters=32, 
            kernel_size=3,
            padding="causal",
            activation="relu",
            input_shape=[N_PAST, 7],
            ),
    Bidirectional(LSTM(32, return_sequences=True)),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(N_FEATURES)
])

# Checkpoint

In [16]:
checkpoint_path='model/my_checkpoint.ckpt'

checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1,
                             )

# Model Compile

In [17]:
# learning_rate=0.0005, Adam 옵치마이저
optimizer =  tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss='mae',
              optimizer=optimizer,
              metrics=["mae"]
              )

# Fit

In [18]:
model.fit(train_set, 
        validation_data=(valid_set), 
        epochs=20, 
        callbacks=[checkpoint], 
        )

Epoch 1/20
   2159/Unknown - 59s 21ms/step - loss: 0.0507 - mae: 0.0507
Epoch 1: val_loss improved from inf to 0.04422, saving model to model/my_checkpoint.ckpt
2159/2159 [==============================] - 71s 26ms/step - loss: 0.0507 - mae: 0.0507 - val_loss: 0.0442 - val_mae: 0.0442
Epoch 2/20
2157/2159 [============================>.] - ETA: 0s - loss: 0.0437 - mae: 0.0437
Epoch 2: val_loss improved from 0.04422 to 0.04351, saving model to model/my_checkpoint.ckpt
2159/2159 [==============================] - 61s 24ms/step - loss: 0.0437 - mae: 0.0437 - val_loss: 0.0435 - val_mae: 0.0435
Epoch 3/20
2157/2159 [============================>.] - ETA: 0s - loss: 0.0423 - mae: 0.0423
Epoch 3: val_loss improved from 0.04351 to 0.04152, saving model to model/my_checkpoint.ckpt
2159/2159 [==============================] - 61s 24ms/step - loss: 0.0423 - mae: 0.0423 - val_loss: 0.0415 - val_mae: 0.0415
Epoch 4/20
2159/2159 [==============================] - ETA: 0s - loss: 0.0414 - mae: 0.0414

In [19]:
model.load_weights(checkpoint_path)

# 모델 검증

In [20]:
model.evaluate(valid_set)

539/539 [==============================] - 13s 15ms/step - loss: 0.0378 - mae: 0.0378


[0.037809886038303375, 0.03780988231301308]